In [1]:
# Import statements

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import random 
from copy import deepcopy

%matplotlib inline
import gym

In [2]:
# Register new environment
gym.envs.register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False}, # True ....
    max_episode_steps=100,
    reward_threshold=0.78, # optimum = .8196
)

In [3]:
# Some basic parameters
inpDir = '../input'
outDir = '../output'

RANDOM_STATE = 42
STEPS = 200

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (9, 6),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

plt.rcParams.update(params)


SMALL_ENOUGH = 1e-9
GAMMA = 0.95

In [4]:
# Helper function to plot the grid

def fn_plot_heatmap(values, policy):
    fig, ax = plt.subplots()
    act = [action_mappings[action] for action in np.argmax(policy, axis=1)]
    labels = np.asarray(["{0}\n{1:.3f}".format(a, v)
                          for a, v in zip(act,values)]).reshape(4,4)
    values = values.reshape(4,4)
    sns.heatmap(values, annot=labels, fmt="", cmap='RdYlGn', ax=ax)
    plt.show()

In [5]:
# Load the frozen lake Environment
grid = gym.make('FrozenLakeNotSlippery-v0')
#grid = gym.make('FrozenLake-v0')
print('Environment Created.')

Environment Created.


In [6]:
grid.reset()
grid.render()


SFFF
FHFH
FFFH
HFFG


In [7]:
print(grid.observation_space)

Discrete(16)


In [8]:
print('Observation Space:' , grid.observation_space.n)

Observation Space: 16


In [9]:
print ('States:', grid.nS)

States: 16


In [10]:
# Randomly sample from our observation space
for i in range(10):
    print(grid.observation_space.sample())

2
4
2
8
14
15
11
3
0
15


In [11]:
# Action mappings - Map actions to numbers
action_mappings = {
    0: '\u2190' , # Left
    1: '\u2193', # Down
    2: '\u2192', # Right
    3: '\u2191', # Up
}

In [12]:
# Action space
print('Action Space:', grid.action_space)

A_n = grid.action_space.n
print('Action:', A_n)


Action Space: Discrete(4)
Action: 4


In [13]:
# Random Action sample

for i in range(10):
    a_n = grid.action_space.sample()
    print('Action: ', a_n, ',  Moving towards : ',action_mappings[a_n] )

Action:  2 ,  Moving towards :  →
Action:  0 ,  Moving towards :  ←
Action:  3 ,  Moving towards :  ↑
Action:  3 ,  Moving towards :  ↑
Action:  3 ,  Moving towards :  ↑
Action:  2 ,  Moving towards :  →
Action:  3 ,  Moving towards :  ↑
Action:  1 ,  Moving towards :  ↓
Action:  0 ,  Moving towards :  ←
Action:  2 ,  Moving towards :  →


In [14]:
# Go Down (action = 1)
grid.reset()
action = 1
grid.render()
observation, reward, done, prob = grid.step(action)
grid.render()
print(observation, reward, done, prob)


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
4 0.0 False {'prob': 1.0}


In [15]:
# Stochastic
env = gym.make('FrozenLakeNotSlippery-v0')

In [16]:
# lets start with random policy
S_n = env.observation_space.n
A_n = env.action_space.n

policy = np.random.rand(S_n,A_n)
policy

array([[0.158071  , 0.20778276, 0.33145394, 0.35882548],
       [0.87680062, 0.42891501, 0.99833698, 0.09145727],
       [0.07209985, 0.86575896, 0.73518778, 0.84662982],
       [0.29919122, 0.99782712, 0.76639611, 0.26200473],
       [0.36607076, 0.17085959, 0.8489023 , 0.57109656],
       [0.12777564, 0.91387493, 0.89487482, 0.33010979],
       [0.91852971, 0.92689572, 0.22044131, 0.80142922],
       [0.00762966, 0.50197827, 0.20288965, 0.11728858],
       [0.11253251, 0.1090981 , 0.13636724, 0.77399484],
       [0.77474249, 0.92138079, 0.61020325, 0.89486855],
       [0.42991518, 0.37171078, 0.04775299, 0.72220491],
       [0.31038534, 0.85091737, 0.38257959, 0.56469743],
       [0.99838945, 0.1674564 , 0.08880521, 0.73517257],
       [0.3579591 , 0.47502698, 0.08359938, 0.72992685],
       [0.17016566, 0.81461506, 0.30229156, 0.25178729],
       [0.66332483, 0.80275122, 0.89039609, 0.26412869]])

In [26]:
epsilon = 0.5
decay=0.9

In [39]:
def epsilon_greedy_action(state, pol, epsilon):

    values = list(pol[state,:]) # 1*4
    max_value = max(values)
    no_actions = len(values)
    
    explore = random.random() < epsilon
  
    if explore:
        return random.choice([0,1,2,3])
    else:
        return values.index(max_value)

In [40]:
def generate_episode(policy, env,epsilon):
    
    # we initialize the list for storing states, actions, and rewards
    states, actions, rewards = [], [], []
    # Initialize the gym environment
    # observation means current state
    observation = env.reset()
    
    while True:
        # append the states to the states list
        states.append(observation)
        
        # now, we select an action using our sample_policy function and append the action to actions list
        epsilon = epsilon*decay
        action = epsilon_greedy_action(observation,policy,epsilon)
        actions.append(action)
        
        # We perform the action in the environment according to our sample_policy, move to the next state 
        # and receive reward
        #print(observation)
        observation, reward, done, info = env.step(action)
        rewards.append(reward)
        
        # Break if the state is a terminal state
        if done:   #hole and gola are terminal states
             break
    #env.render()            
    return states, actions, rewards 

In [41]:
def first_visit_mc_prediction(policy, env, n_episodes,epsilon):
    
    # First, we initialize the empty value table as a dictionary for storing the values of each state
    value_table = defaultdict(float)
    N = defaultdict(int)
    #states = [i for i in range(16)]
    #observation = random.choice(states)
    
    for _ in range(n_episodes):
        
        epsilon = epsilon*decay
        # Next, we generate the epsiode and store the states and rewards
        states, actions, rewards = generate_episode(policy, env, epsilon)
        returns = 0
        
        # Then for each step, we store the rewards to a variable R and states to S, and we calculate
        # returns as a sum of rewards
        
        for t in range(len(states) - 1, -1, -1):
            R = rewards[t]
            S = states[t]
            A = actions[t]
            returns += R
            
            # Now to perform first visit MC, we check if the episode is visited for the first time, if yes,
            # we simply take the average of returns and assign the value of the state as an average of returns
            
            if S not in states[:t]: #not considering repeated states
                N[S] += 1
                value_table[S] += (returns - value_table[S]) / N[S]

    return value_table

In [42]:
value = first_visit_mc_prediction(policy, env,5000,epsilon)

In [43]:
value

defaultdict(float, {4: 0.0, 0: 0.0, 10: 0.0, 6: 0.0, 2: 0.0, 1: 0.0, 14: 0.0})

In [45]:
policy

array([[0.158071  , 0.20778276, 0.33145394, 0.35882548],
       [0.87680062, 0.42891501, 0.99833698, 0.09145727],
       [0.07209985, 0.86575896, 0.73518778, 0.84662982],
       [0.29919122, 0.99782712, 0.76639611, 0.26200473],
       [0.36607076, 0.17085959, 0.8489023 , 0.57109656],
       [0.12777564, 0.91387493, 0.89487482, 0.33010979],
       [0.91852971, 0.92689572, 0.22044131, 0.80142922],
       [0.00762966, 0.50197827, 0.20288965, 0.11728858],
       [0.11253251, 0.1090981 , 0.13636724, 0.77399484],
       [0.77474249, 0.92138079, 0.61020325, 0.89486855],
       [0.42991518, 0.37171078, 0.04775299, 0.72220491],
       [0.31038534, 0.85091737, 0.38257959, 0.56469743],
       [0.99838945, 0.1674564 , 0.08880521, 0.73517257],
       [0.3579591 , 0.47502698, 0.08359938, 0.72992685],
       [0.17016566, 0.81461506, 0.30229156, 0.25178729],
       [0.66332483, 0.80275122, 0.89039609, 0.26412869]])

In [46]:
n_iteration = 1000
highscore = 0
data_row = []
for n_iter in range(n_iteration):

    points = 0 # keep track of the reward each episode
    data_dict = {}
    for i_episode in range(20): # run 20 episodes
        print ('_'*50)
        observation = env.reset()
        while True: # run until episode is done
            #grid.render()
            action = np.argmax(policy[observation])
            observation, reward, done, info = env.step(action)
            points += reward
            if done:
                env.render()
                #print (observation, reward, done, info)

                if points > highscore: # record high score
                    highscore = points
                break
    data_dict['run'] = n_iter
    data_dict['Points']=  points
    data_row.append(data_dict)

__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
F

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
F

__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
F

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

...

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
____________________________________________

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
__________________________________________________
  (Up)
SFFF
FHFH
FFFH
HFFG
_____________________________________

In [ ]:
print(highscore)